# Yelp API Package

# Import necessary libraries

In [11]:
# imports
import pandas as pd
import json
from yelpapi import YelpAPI

In [4]:
# instantiate
with open('/Users/shell/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [5]:
# create an instance with my key
yelp_api = YelpAPI(login['API Key'], timeout_s = 5.0)
yelp_api

* To use the "businesses search" endpoint we will use the yelp_api.search_query method.
    * If we inspect the docstring for the function (either run the help function on it or place your cursor inside the parenthesis for it and hit Shift+Tab), we see that it doesn't tell us very much.

In [6]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



* The only arguments/parameters that the docstring mentions are location or latitude & longitude.
* Notice that the link to the official API documentation is linked instead.
    * This is because this documentation only shows what the required parameters are, but it will accept ANY of the parameters that are accepted by the business_search endpoint.

In [7]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location = 'NY, NY',
                                                  term = 'Pizza')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

* The package returns the results in the JSON format we have been exploring. Note that the exact results may vary as the Yelp site is constantly changing.

In [8]:
# search results
search_results['total']

12300

In [12]:
# load data
biz = pd.DataFrame(search_results['businesses'])
biz.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/AaCoCI...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4597,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/HB5-Bo...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2559,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844


# Pagination and "offset"
* Pagination can be understood through this brief demonstration:

* Total Results vs Businesses

    * If we check the total number of results, we can see that we had 12,000 businesses that met our search criteria.

In [13]:
# total number of matching businesses
search_results['total']

12300

In [14]:
# how many businesses in our results?
len(search_results['businesses'])

20

* We have 12000 results, but the length is only 2.? What's going on?

* If you think about the results of a google search, we usually have MANY more results that Google will display at one time. When we scroll down to the bottom of the results, we can see that Google has divided the results into several PAGES of results instead of one extremely long page.

* This is what Yelp Fusion is doing as well! The general term for this is "Pagination".

* The Yelp API will only return a "page" of 20 results at a time.

    * If we want to get the next page of results, we will perform another API call, but we will add an additional argument called "offset".
        * The offset is what # result to use as the FIRST result for the page.
        * If we had 20 businesses in our first result, we would want to add an offset of 20.


In [17]:
# add offset to out original API call 
search_results = yelp_api.search_query(location = 'NY, NY',
                                       term = 'Pizza',
                                       offset = 20)

* Now check the results.

In [18]:
biz20 = pd.DataFrame(search_results['businesses'])
biz20.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,Ii_CFGFaTUpfGH1xOUZXVQ,rosie-pizza-bar-new-york,Rosie Pizza Bar,https://s3-media4.fl.yelpcdn.com/bphoto/4b8xU9...,False,https://www.yelp.com/biz/rosie-pizza-bar-new-y...,37,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.6998, 'longitude': -73.92842}","[pickup, delivery]","{'address1': '128 Central Ave', 'address2': ''...",+17184404369,(718) 440-4369,5588.922717,NaN
1,hGksyTiHBBdrhbiVjXzWVw,scarrs-pizza-new-york,Scarr's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/xonxGJ...,False,https://www.yelp.com/biz/scarrs-pizza-new-york...,638,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71534, 'longitude': -73.9914}","[pickup, delivery]","{'address1': '22 Orchard St', 'address2': '', ...",+12123343481,(212) 334-3481,1125.784330,$


* You should have different results than your original call when you make the offset call.

* Now you can combine the results (so far) into one dataframe using pd.concat()

In [19]:
# concatenate the previous results and the new results
businesses = pd.concat([biz, biz20], ignore_index = True)
display(businesses.head(3), businesses.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/AaCoCI...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4597,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/HB5-Bo...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2559,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/fisdCw...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,811,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,atbANdSznV29GI066I2Ijw,champion-pizza-new-york-6,Champion Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SBZL50...,False,https://www.yelp.com/biz/champion-pizza-new-yo...,403,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7215270996094, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '17 Cleveland Pl', 'address2': No...",+12122263777,(212) 226-3777,1801.476444
38,xxgcU_oFIox_pAmroo9WwA,front-street-pizza-brooklyn,Front Street Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/ybrfmg...,False,https://www.yelp.com/biz/front-street-pizza-br...,436,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.70244, 'longitude': -73.98943}","[pickup, delivery]",$,"{'address1': '80 Front St', 'address2': '', 'a...",+17188753700,(718) 875-3700,550.439226
39,i_rg-OLXQbpRUN7O3MeCeA,piz-zetta-brooklyn-2,Piz-zetta,https://s3-media1.fl.yelpcdn.com/bphoto/VmaEn7...,False,https://www.yelp.com/biz/piz-zetta-brooklyn-2?...,322,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.69116658201196, 'longitude': -...","[pickup, delivery]",$,"{'address1': '90 Livingston St', 'address2': '...",+17184227878,(718) 422-7878,1616.959411


# Summary
* Making API calls requires our API Key and knowledge of the endpoints we want to use.
* Reading the documentation is particularly important for using APIs.
* The output of your API call may be broken into bite-sized chunks known as pagination. You will need to use offset to access all of the relevant information.
* The results can then be concatenated into a single data frame.